<a href="https://colab.research.google.com/github/bc1414/Machine-Learning-Notes/blob/main/Closeness-and-degree-analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""

Measures the correlation between degree and closness

Based on
Linking the Network Centrality Measures：  Closeness and Degree

Tim Evans, Bingsheng Chen

Implementation and interpreation by Bingsheng Chen(Imperial College London)

@author: Bingsheng Chen
"""


In [ ]:
import igraph
from igraph import *
import networkx as nx
import numpy as np
import scipy
import scipy as sp
import scipy.stats
import matplotlib as plt
import os
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import metrics
from uncertainties import ufloat
from uncertainties.umath import *
from uncertainties import unumpy



In [ ]:



def farness_and_degree(G):
  '''
    Calculate farness (1/closeness) and degree for a graph for all nodes.

    Input
    -----
    G -- a simple, undirected, unweighted graph-tool graph of one component.

    Return
    -----
    k -- degree of nodes.
    farness -- inverse closeness of nodes.
  '''

  c=np.array(G.closeness())
  k=np.array(G.degree())
  c= c[~np.isnan(c)]
  k=k[k>0]
  farness=1/c
  return k,farness



def std_for_farness(farness,k):
  '''
  Bin nodes of same degree for calculating
  average farness and standard deviation of farness. Those values
  are important for goodness of fit test. Ignore degrees with only
  one farness value.

  NOTE:- this is the standard deviation for each value of farness based
  on all the other farness values at teh same degree.  It is an estimate of the
  error in a single measurement of farness so it makes sense to return
  this array which will contain the value whenever the degree is the same.
  That is if k[i]=k[j] then std_for_farness[i]=std_for_farness[j]
  The mean farness at each degree is not calculated here.
  However if we did calculate the mean farness then we would need to find the
  errror in the mean.

  Input
  ----
  farness -- an array of inverse of closeness
  k    -- an array of degree


  Output
  ----
  std_farness_k -- an array (length N) of standard deviation of
          farness of to every node based on the degree k.
  '''
  k_list=np.unique(k)
  std_farness=np.ones(len(k))
  for degree in k_list:
      mask=k==degree
      std_farness[mask]=np.std(farness[mask])
  return std_farness


def reduced_chisquared(obs,std,fit,tol=0.001):
    '''
    Calculate Reduced chi-squared statistics.

    See Numerical Recipes section 15.1.1 Chi-Square Fitting, equation 15.1.6
    http://numerical.recipes/book/book.html

    Assumes there are two constraints so that the number of degrees of freedom
    is two less than the number of observations used (those where std>tol)

    Input
    ----
    obs -- an array of observed data, here it means measured
        farness from the Graph
    std -- an array of standard deviation of farness for each
        observed farness
    fit -- an array of predicted result from the model.
    tol=0.001 -- tolerance, only include terms where std is greater than tol

    Output
    ----
    rc -- reduced chi-squre statistics.
    '''
    # TSE x**2 is likely to be slow. Better to write x*x if you can
    chi_square=np.sum(((obs[std>tol]-fit[std>tol])/std[std>tol])**2)
    rc=chi_square/(len(obs[std>tol])-2) #first order fit N-1-1, i.e. two constraints
    return rc


def fitting_model(k,farness,std_farness):
    '''
    Determine the slope and intercept term between inverse closeness and logarithm
    of degree using least-square fit.

    Input
    ----
    k            -- an array of degree in the graph
    farness      -- an array of inverse closeness
    std_farness  -- an array of standard deviation of farness

    Output
    ----
    rc           -- float, reduced chi-squaared statistics
    slope        -- float, the slope term
    slope_err    -- float, error in finding slope term
    beta         -- float, intercept term
    beta_err     -- float, error in finding intercept term
    '''
    lnk=np.log(k)
    """
    Note as we are using raw data not the mean value at each degree value,
    there is no obvious need to weight (e.g. by inverse std.dev.)
    and the default is to weight each value equally
    https://numpy.org/doc/stable/reference/generated/numpy.polyfit.html
    """
    #fitting to farness[i] gives coef[0]*lnk[i] + coef[1]
    coef,V=np.polyfit(lnk, farness, 1, cov=True)
    fitted_model=np.poly1d(coef)
    fitted_farness=fitted_model(lnk)
    rc=reduced_chisquared(farness,std_farness,fitted_farness)
    slope=coef[0]
    beta=coef[1]
    slope_err=np.sqrt(V[0][0])
    beta_err=np.sqrt(V[1][1])
    return rc,slope,slope_err,beta,beta_err





def effective_branching_and_beta_t(slope,slope_err,N):
  '''
  Calculating effective branching number(zbar) from the slope term and also use
  the effective branching number to determine the intercept term.

  Error evaulation is based on uncertainty package, for more details,
  see https://pythonhosted.org/uncertainties/.


  Input
  ----
  slope    -- float, slope term calculating from the inverse closeness and log k.
  sloe_err -- float, error term in determining slope.
  N        -- float, number of vertices in the network.

  Output
  ----
  slope_u  -- uncertainity float, slope term with error
  zbar_u   -- uncertainity float, effective branching number with error
  beta_t_u -- uncertainity float, intercept term predict by effective branching


  '''
  slope_u = ufloat(slope, slope_err)
  zbar_u=unumpy.exp(-1/slope_u)
  beta_t_u=(1/(zbar_u-1)+unumpy.log(zbar_u-1)/unumpy.log(zbar_u))+unumpy.log(N)/unumpy.log(zbar_u)

  return slope_u,zbar_u,beta_t_u

def avg_shortest_path_prediction(slope_u,beta_t_u,beta,beta_err,avg_lnk):
  '''
  Calculate average shortest path in the network using the effective
  branching number and intercept term using fitted intercept term
  and estimated intercept term using effective branching number.



  Input
  ----
  slope_u  -- uncertainity float, slope
  beta_t_u  --  uncertainity float, intercept estimated by effective
          branching number
  beta    -- float, intercept term
  beta_err  -- float error in intercept term
  avg_lnk   -- float, average of \ln k

  ----
  Output
  avg_path_fit_u -- uncerntainty float, avearge shortest path calculated
            using fitted intercept term
  avg_path_pred_u -- uncertainty float, average shortest path computed
            using intercept term predicted by effective
            branching number.

  '''

  beta_u=ufloat(beta,beta_err)
  avg_path_fit_u=+slope_u*avg_lnk+beta_u
  avg_path_pred_u=avg_lnk*slope_u+beta_t_u
  return avg_path_fit_u,avg_path_pred_u




def evaluate_closeness_and_degree_relation(G):
  '''
  Analyze the graph to see if the degree and closeness are correlated.
  This can indicate whether the structure behind the
  network can be approximated by a shortest-path tree have a finite
  braching behavior. The function will automatically extract LCC first
  and remove parallel edges and self loop. In addition, the graph
  will be converted to undirected.


  Input
  ----
  G     -- a graph-tool graph

  ----
  Output
  Report -- pandas dataframe contains summary about the network

  '''
  k,farness=farness_and_degree(G)
  avg_lnk=np.mean(np.log(k))
  N=len(k)
  avg_path=np.mean(farness)
  std_farness=std_for_farness(farness,k)
  rc,slope,slope_err,beta,beta_err=fitting_model(k,farness,std_farness)
  slope_u,zbar_u,beta_t_u=effective_branching_and_beta_t(slope,slope_err,N)
  beta_u=ufloat(beta,beta_err)
  avg_path_fit_u,avg_path_pred_u=avg_shortest_path_prediction(slope_u,beta_t_u,beta,beta_err,avg_lnk)
  rho_inv,pvalue=np.corrcoef(np.log(k),farness)[:,1]
  rho,p=sp.stats.pearsonr(k,1/farness)
  report=(N,np.mean(k),rc,zbar_u,beta_u,beta_t_u,rho,rho_inv,avg_path,avg_path_fit_u,avg_path_pred_u)
  return report


In [ ]:
# Social_Gl=np.load('Social_igraph.npy',allow_pickle=True)
# Commun_Gl=np.load('Commun_igraph.npy',allow_pickle=True)
# Hpl_Gl=np.load('Hpl_igraph.npy',allow_pickle=True)
# Coauth_Gl=np.load('Coauth_igraph.npy',allow_pickle=True)
# Citation_Gl=np.load('Citation_igraph.npy',allow_pickle=True)
# print(Citation_Gl)
# Comb_Gl=np.hstack([Social_Gl,Commun_Gl,Coauth_Gl,Citation_Gl,Hpl_Gl])



In [ ]:
# Comb_Gl

array([<igraph.Graph object at 0x00000269D1A068B0>,
       <igraph.Graph object at 0x00000269D1CC54F0>], dtype=object)

In [ ]:
# evaluate_closeness_and_degree_relation(Comb_Gl[0])

(34,
 4.588235294117647,
 1.2309594865490001,
 array(8.805519214156051+/-2.761955155749791, dtype=object),
 2.996824805911752+/-0.09528648867777845,
 2.693732856465253+/-0.25260254807344207,
 0.7715909962817317,
 -0.7749194336723219,
 2.408199643493761,
 2.4081996434937603+/-0.12760411446048114,
 2.1051076940472617+/-0.1677303199731785)

In [ ]:
def auto_graph_stats(G_list):
  df=[]
  i=0
  for G in G_list:
     i+=1
     print(i)
     report=evaluate_closeness_and_degree_relation(G)
     df.append(report)
  df = pd.DataFrame(df, columns=[r" $|V|$ ", r"<k>", r"Reduced $\chi^2$",r"$\bar{z}$",r"$\hat{\beta}$",r"$\beta_t$",r"$\rho$",r" Improved $\rho$",r"avg path length",r"avg path length predicted via fit",r"avg path length predicted via $\bar{z}$"])
  return df
df=auto_graph_stats(Comb_Gl)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18


In [ ]:
df[r" $|V|$ "]

0        34
1       198
2      2539
3      1788
4        70
5       217
6      1133
7      1893
8       986
9      1833
10    29652
11    14845
12      379
13     6927
14    12494
15    23166
16     1222
17     1222
Name:  $|V|$ , dtype: int64